In [ ]:
import tensorflow as tf

In [ ]:
import sys
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
from util import expand_nodelist
import os

In [ ]:
def setup_slurm_cluster():
    tf_hostlist = [ ("%s:22222" % node) for node in
                expand_nodelist( os.environ['SLURM_NODELIST']) ] 
    ps_hosts = [tf_hostlist[0]]
    print ps_hosts
    worker_hosts = tf_hostlist[1:]
    print worker_hosts
    task_index  = int( os.environ['SLURM_PROCID'] )
    n_tasks     = int( os.environ['SLURM_NPROCS'] )
    
    cluster = tf.train.ClusterSpec({
    "worker": worker_hosts,
    "ps": ps_hosts
    })

    if task_index == 0:
        job_name = "ps"
    else:
        job_name = "worker"
        #expects a task_index for worker
        task_index -= 1
    # Create and start a server for the local task.
    server = tf.train.Server(cluster,
                           job_name=job_name,
                           task_index=task_index)
    return cluster, server, task_index, n_tasks, job_name
    

def setup_local_cluster():
    port_list = range(8997,8999)
    tf_hostlist = [ (os.environ["HOSTNAME"] + ":%i" % port) for port in
                port_list]
    
    ps_hosts = [tf_hostlist[0]]
    print ps_hosts
    worker_hosts = tf_hostlist[1:]
    print worker_hosts
    job_name = sys.argv[2]
    task_index  = int(sys.argv[3])
    n_tasks     = len(port_list)
    
    cluster = tf.train.ClusterSpec({
    "worker": worker_hosts,
    "ps": ps_hosts
    })
    
    # Create and start a server for the local task.
    server = tf.train.Server(cluster,
                           job_name=job_name,
                           task_index=task_index)
    

    return cluster, server, task_index, n_tasks, job_name

def setup_cluster(typ_):
    if typ_ == "slurm":
        return setup_slurm_cluster()
    else:
        return setup_local_cluster()